In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re


# URL to scrapp
base_url = "https://www.autosphere.fr"
# All electric
search_url_template = "https://www.autosphere.fr/recherche?fuel_type=Electrique&from={}"
# Pour un modèle en particulier
search_url_model = "https://www.autosphere.fr/recherche?fuel_type=Electrique&brand=Peugeot&model=2008"

# la pagination d'autosphère se fait via un from et avance de 23 en 23 pour le moment
all_links = set()
offset = 2576 # -> start
step = 23
infos = {} # Pour le df de fin a exporter dans l'excel 


### On récupère tous les liens en parcourant toutes les pages

while True:
    url = search_url_template.format(offset)
    print(f"Scraping page avec from={offset} ...")
    response = requests.get(url)

    # S'arrête lorsque que la page n'a pas de lien
    if response.status_code != 200:
        print(f"Page avec from={offset} inaccessible. Fin du scraping.")
        break

    # Structure des lien des véhicules -> fiche ou fiche4 ou fiche-mixte puis / auto-occasion
    soup = BeautifulSoup(response.text, 'html.parser')
    new_links = set()
    for a_tag in soup.select('a[href*="/fiche"]'):
        href = a_tag.get('href')
        if href and "/auto-occasion" in href:
            full_url = base_url + href
            new_links.add(full_url)

    if not new_links:
        print(f"Aucun nouveau lien trouvé avec from={offset}. Fin du scraping.")
        break

    initial_count = len(all_links)
    all_links.update(new_links)
    added_count = len(all_links) - initial_count

    print(f"{added_count} nouveaux liens trouvés avec from={offset}.")
    offset += step
    time.sleep(1)

print(f"\nNombre total de fiches uniques récupérées : {len(all_links)}")

### Récupération des infos pour chaque véhicule
car_nbr = 1
for link in all_links:
    infos[car_nbr] = {}
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")  # Active cette ligne si tu veux en mode sans fenêtre
    driver = webdriver.Chrome(options=options)

    driver.get(link)
    try:
        wait = WebDriverWait(driver, 15)

        all_li = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//li")))

        score_sante = None
        kilometrage = None
        annee = None

        for li in all_li:
            text = li.text.strip()

            if "Santé" in text and not score_sante:
                match = re.search(r"(\d+\s?%)", text)
                if match:
                    score_sante = match.group(1)

            if "km" in text and not kilometrage:
                match = re.search(r"([\d\s]+km)", text)
                if match:
                    kilometrage = match.group(1).strip().replace("\u202f", "").replace(" ", "").replace("km", "").strip()

            if re.search(r"\b20\d{2}\b", text) and not annee:
                annee = re.search(r"\b(20\d{2})\b", text).group(1)

        # Titre principal de la fiche (h1)
        titre_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h1")))
        titre_text = titre_element.text.strip()

        # Version courte si détectée dans le titre
        version = None
        match = re.search(r"\d+ch\s+\w+", titre_text)
        if match:
            version = match.group(0)

        # === Récupération de la marque, modèle, version complète via fil d’Ariane ===
        marque = None
        modele = None
        version_complete = None

        try:
            links_breadcrumb = wait.until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "li.inline-flex.items-center a.text-blue-700")
            ))
            if len(links_breadcrumb) >= 2:
                marque = links_breadcrumb[2].text.strip()
                modele = links_breadcrumb[3].text.strip()
            version_span = driver.find_element(By.CSS_SELECTOR, "span.text-black-600")
            version_complete = version_span.text.strip()
            if version_complete.find(' - ') > 0:
                version_complete = version_complete[:version_complete.find('-')]
            if modele !="2008":
                version_complete = re.sub(r"\b20\d{2}\b", "",  version_complete)
            version_complete = version_complete.strip()

        except Exception as e:
            print("Erreur lors de la récupération marque/modèle/version:", e)

        # Enregistrement pour l'excel 
        
        infos[car_nbr]["lien"] = link
        infos[car_nbr]['SoH'] = score_sante
        infos[car_nbr]['Année'] = annee
        infos[car_nbr]['Odomètre (km)'] = kilometrage
        infos[car_nbr]['Type'] = version_complete
        infos[car_nbr]['Modèle'] = modele
        infos[car_nbr]['OEM'] = marque

        car_nbr +=1
    finally:
        driver.quit()


In [ ]:
pd.DataFrame(infos).T.dropna(subset='SoH')

In [ ]:
infos

In [ ]:
import pandas as pd
pd.DataFrame(infos).T[["OEM","Modèle","Type","Année","Odomètre (km)","SoH"]].dropna(subset='SoH')

In [ ]:
infos

In [ ]:
from scrapping.scrapping_autospherre import export_to_excel

In [ ]:
from transform.insights_results.trendlines_excel import get_gspread_client
def export_to_excel(df_to_write, gsheet):
    client = get_gspread_client()
    sheet_out = client.open("202505 - Courbes SoH")
    worksheet = sheet_out.worksheet(gsheet)
    worksheet.append_rows(df_to_write.values.tolist())
    print(f"Données écritent dans {gsheet}")

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({1: {'lien': 'https://www.autosphere.fr/fiche/auto-occasion-dacia-spring-45ch-business-2020-achat-integral-62000-arras-450331',
  'SoH': None,
  'Année': '2020',
  'Odomètre (km)': '31764',
  'Type': '45ch Business',
  'Modèle': 'Spring',
  'Marque': 'Dacia',
  'Version complète': '45ch Business'},}).T[["OEM","Modèle","Type","Année","Odomètre (km)","SoH"]]

In [ ]:
export_to_excel(df, 'Courbes OS')